In [65]:
import numpy as np
import pandas as pd
import re
import nltk
import collections
import math
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from unicodedata import normalize
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samanthakem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
DATA_PATH = 'data/estadao_noticias_eleicao.csv'
QUANTIDADE_ROWS = 10
VALOR_AXIS = 1
NOTICIA_COMPLETA_COL = 'noticia'
TITULO_COL = 'titulo'
SUBTITULO_COL = 'subTitulo'
CONTEUDO_COL = 'conteudo'
ID_NOTICIA_COL = 'idNoticia'
URL_COL = 'url'
TOKENS_COL = 'tokens'

In [67]:
# MÉTODOS AUXILIARES

In [68]:
def concatena_noticia(row):
    return clean((row[TITULO_COL] + " " + row[SUBTITULO_COL] + " " + row[CONTEUDO_COL]).lower())

def tokeniza(conteudo):
    stop_words = set(stopwords.words('portuguese'))
    return set([w for w in word_tokenize(conteudo) if not w in stop_words])

def clean(txt):
    return normalize('NFKD', re.sub(r'\W+', ' ', txt)).encode('ASCII', 'ignore').decode('ASCII')

def criar_indexes(tokens, docId):
    for token in tokens:
        if token in dictionary:
            if docId in dictionary[token]:
                dictionary[token][docId] += 1
            else:
                dictionary[token][docId] = 1
        else:
            dictionary[token] = {}
            dictionary[token][docId] = 1

def calcular_idf(token):
    M = len(df.noticia)
    k = len(dictionary[token].keys())
    return math.log((M + 1) / k)

def criar_idf(token):
    idf = calcular_idf(token)
    idf_dictionary[token] = idf


In [69]:
# MODELOS VETORIAIS

In [62]:
def representacao_binaria(query):
    f_query = query.lower().split(' ')
    aux1 = {}
    aux2 = {}
    
    for docId in dictionary[f_query[0]]: aux1[docId] = 1
        
    for word in f_query[1:]:
        for docId in dictionary[word]:
            if docId in aux1: aux2[docId] = 1

    return [doc[0] for doc in [(k, aux2[k]) for k in sorted(aux2, key=aux2.get, reverse=True)][:10]]

def tf(query):
    f_query = query.lower().split(' ')
    aux1 = {}
    aux2 = {}

    for docId in dictionary[f_query[0]]: aux1[docId] = dictionary[f_query[0]][docId]
    
    for word in f_query[1:]:   
        for docId in dictionary[word]:
            if docId in aux1: aux2[docId] = aux1[docId] + dictionary[word][docId]

    return [doc[0] for doc in [(k, aux2[k]) for k in sorted(aux2, key=aux2.get, reverse=True)][:10]]

In [47]:
dictionary = collections.defaultdict(list)

idf_dictionary = {}

df = pd.read_csv(DATA_PATH)
df = df.replace(np.NAN, "")

df.head(n=QUANTIDADE_ROWS)

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5
5,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",6
6,2014-12-31T00:00:00Z,Veja os desafios dos governadores que assumem ...,,"No Acre, governador reeleito quer erradicar an...","http://politica.estadao.com.br/noticias/geral,...",7
7,2014-12-31T00:00:00Z,PT impulsiona cerimônia de posse da Dilma nas ...,"Já entre os 27 governadores eleitos, apenas se...","Os perfis da presidente Dilma Rousseff, nas re...","http://politica.estadao.com.br/noticias/geral,...",8
8,2014-12-31T00:00:00Z,,"Em setembro de 2014, após conceder entrevista ...",,"http://tv.estadao.com.br/videos,politica,dilma...",9
9,2014-12-31T00:00:00Z,Dilma veta projeto de Suplicy que cria linha o...,Proposta havia sido aprovada no Senado depois ...,A presidente Dilma Rousseff vetou integralment...,"http://politica.estadao.com.br/noticias/geral,...",10


In [48]:
df[NOTICIA_COMPLETA_COL] = df.apply(lambda row: concatena_noticia(row), axis=VALOR_AXIS)

In [49]:
df = df[[ID_NOTICIA_COL, NOTICIA_COMPLETA_COL, URL_COL]]

In [50]:
df.head(n=QUANTIDADE_ROWS)

,idNoticia,noticia,url
0,1,pt espera 30 mil pessoas em festa na esplanada...,"http://politica.estadao.com.br/noticias/geral,..."
1,2,alckmin toma posse de olho no planalto governa...,"http://politica.estadao.com.br/noticias/geral,..."
2,3,seis obstaculos e desafios do segundo mandato ...,"http://politica.estadao.com.br/noticias/geral,..."
3,4,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil..."
4,5,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil..."
5,6,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil..."
6,7,veja os desafios dos governadores que assumem ...,"http://politica.estadao.com.br/noticias/geral,..."
7,8,pt impulsiona cerimonia de posse da dilma nas ...,"http://politica.estadao.com.br/noticias/geral,..."
8,9,em setembro de 2014 apos conceder entrevista ...,"http://tv.estadao.com.br/videos,politica,dilma..."
9,10,dilma veta projeto de suplicy que cria linha o...,"http://politica.estadao.com.br/noticias/geral,..."


In [51]:
df[TOKENS_COL] = df.apply(lambda row: tokeniza(row[NOTICIA_COMPLETA_COL]), axis=VALOR_AXIS)

In [52]:
df.head(n=QUANTIDADE_ROWS)

,idNoticia,noticia,url,tokens
0,1,pt espera 30 mil pessoas em festa na esplanada...,"http://politica.estadao.com.br/noticias/geral,...","{49, pro, chefes, brasilia, ajudaria, caravana..."
1,2,alckmin toma posse de olho no planalto governa...,"http://politica.estadao.com.br/noticias/geral,...","{representa, 25, pavimentacao, internos, posto..."
2,3,seis obstaculos e desafios do segundo mandato ...,"http://politica.estadao.com.br/noticias/geral,...","{crescimento, brasilia, ministerial, contempla..."
3,4,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil...","{estadao, fotos, eventos, veja, dia, mundo, es..."
4,5,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil...","{estadao, fotos, eventos, veja, dia, mundo, es..."
5,6,veja as principais fotos do dia e dos eventos...,"http://fotos.estadao.com.br/fotos/politica,dil...","{estadao, fotos, eventos, veja, dia, mundo, es..."
6,7,veja os desafios dos governadores que assumem ...,"http://politica.estadao.com.br/noticias/geral,...","{48, entretanto, elcio, governadores, inicio, ..."
7,8,pt impulsiona cerimonia de posse da dilma nas ...,"http://politica.estadao.com.br/noticias/geral,...","{redes, chefes, chamando, transmissao, percorr..."
8,9,em setembro de 2014 apos conceder entrevista ...,"http://tv.estadao.com.br/videos,politica,dilma...","{apresentou, conceder, estado, grupo, parte, s..."
9,10,dilma veta projeto de suplicy que cria linha o...,"http://politica.estadao.com.br/noticias/geral,...","{conter, erradicacao, bens, 300, relativa, 500..."


In [53]:
for index, row in df.iterrows():  
    criar_indexes(row.tokens, row.idNoticia)

In [54]:
for token in dictionary:
    criar_idf(token)

In [63]:
queries = ['segundo turno', 'lava jato', 'projeto de lei', 'compra de voto', 'ministério público']

resultado_rb = []
resultado_tf = []

for query in queries:
    resultado_rb.append(representacao_binaria(query))
    resultado_tf.append(tf(query))

In [64]:
print (resultado_rb)
print (resultado_tf)

[[1, 7, 13, 26, 69], [3, 13, 15, 27, 43], [7, 10, 25, 38, 56], [82, 347, 553, 748, 854], []]
[[1, 7, 13, 26, 69], [3, 13, 15, 27, 43], [7, 10, 25, 38, 56], [82, 347, 553, 748, 854], []]
